<a id="tpo"></a>

### TPOT(Tree-Based Pipeline Optimization Tool)

[TPOT](http://epistasislab.github.io/tpot/) es una herramienta de optimización de pipelines basada en árboles que utiliza algoritmos genéticos para optimizar las pipelines de aprendizaje automático. TPOT está construido sobre scikit-learn y utiliza sus propios métodos de regresión y clasificación. TPOT explora miles de posibles posibilidades y selecciona la que mejor se ajusta a los datos.

TPOT no puede procesar automáticamente entradas de lenguaje natural. Además, tampoco es capaz de procesar cadenas categóricas, que deben ser codificadas con números enteros antes de ser pasadas como datos.


![](https://res.cloudinary.com/dyd911kmh/image/upload/f_auto,q_auto:best/v1537396029/output_2_0_d7uh0v.png)


## Instalación

La instalación es bastante sencilla ya que se encuentra disponible a traves de pip.

In [1]:
#conda install numpy scipy scikit-learn pandas joblib pytorch
#pip install deap update_checker tqdm stopit xgboost
!pip install tpot

     |████████████████████████████████| 87 kB 4.9 MB/s eta 0:00:011
     |████████████████████████████████| 166.7 MB 74.3 MB/s eta 0:00:01
     |████████████████████████████████| 157 kB 43.0 MB/s eta 0:00:01
  Created wheel for stopit: filename=stopit-1.1.2-py3-none-any.whl size=11955 sha256=0e53c6caa56e24368953ade0746b8ce46d148b431eacbf13e452666babe9a536
  Stored in directory: /home/jovyan/.cache/pip/wheels/a8/bb/8f/6b9328d23c2dcedbfeb8498b9f650d55d463089e3b8fc0bfb2
Successfully built stopit


## Pruebas
Como en los casos anteriores se va a realizar pruebas con el dataset de Titanic, ejemplos parecido a este se puede encontrar en la página oficial de [TPOT](http://epistasislab.github.io/tpot/examples/)

Como se ha comentado, TPOT no acepta valores no numéricos como entrada, por lo que hay que realizar una serie de transformaciones al dataset antes de poder utilizarse.

In [1]:
import numpy as np
import pandas as pd
import os        
import sklearn
from sklearn.model_selection import train_test_split        
df = pd.read_csv("./titanic/train.csv") 
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Si analizamos los campos que no pertenecen a este tipo de formato vemos que en el caso de Sex y Embarked se podría realizar un preprocesado previo.

In [2]:
for cat in ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']:
    print("Number of levels in category '{0}': \b {1:2.2f} ".format(cat, df[cat].unique().size))

Number of levels in category 'Name':  891.00 
Number of levels in category 'Sex':  2.00 
Number of levels in category 'Ticket':  681.00 
Number of levels in category 'Cabin':  148.00 
Number of levels in category 'Embarked':  4.00 


In [3]:
for cat in ['Sex', 'Embarked']:
    print("Levels for catgeory '{0}': {1}".format(cat, df[cat].unique()))

Levels for catgeory 'Sex': ['male' 'female']
Levels for catgeory 'Embarked': ['S' 'C' 'Q' nan]


Modificamos manualmente estos valores a valores numéricos. 

In [4]:
df['Sex'] = df['Sex'].map({'male':0,'female':1})
df['Embarked'] = df['Embarked'].map({'S':0,'C':1,'Q':2})

El primer paso es renombrar el feature que se usará como target a class.
Los valores nan, son simplemente remplazados por el valor -999.

In [5]:
df.rename(columns={'Survived': 'class'}, inplace=True)
df = df.fillna(-999)

Dado que Name y Ticket tienen varios posibles valores, los eliminamos de nuestro análisis por simplicidad. Para Cabin, codificamos los niveles como dígitos utilizando el MultiLabelBinarizer de Scikit-learn y los tratamos como nuevas características.

In [6]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
CabinTrans = mlb.fit_transform([{str(val)} for val in df['Cabin'].values])

In [7]:
CabinTrans

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [8]:
df_new = df.drop(['Name','Ticket','Cabin','class'], axis=1)
df_new = np.hstack((df_new.values,CabinTrans))


In [9]:
np.isnan(df_new).any()

False

Por último, almacenamos la feature class, que necesitamos predecir, en una variable separada.

In [10]:
titanic_class = df['class'].values

In [11]:
training_indices, validation_indices = training_indices, testing_indices = train_test_split(df.index, stratify = titanic_class, train_size=0.75, test_size=0.25)
training_indices.size, validation_indices.size

(668, 223)


El package TPOTClassifier tiene una gran variedad de parámetros. Pero los más notables son:

- generations: Número de iteraciones para el proceso de optimización de la pipeline de ejecución. El valor por defecto es 100.

- population_size: Número de individuos por generación. El valor por defecto es 100.

- offspring_size: Número de descendientes a producir en cada generación de programación. El valor por defecto es 100.

- mutation_rate: Tasa de mutación para el algoritmo de programación en el rango [0,0, 1,0]. Este parámetro le dice al algoritmo cuántos cambios aleatorios debe de aplicar en cada generación. Por defecto es 0.9

- crossover_rate: Tasa de cruce para cada generación en el rango [0,0, 1,0]. Este parámetro indica cuántas tuberías debe de generar en cada generación.

- scoring: Función utilizada para evaluar la calidad de una pipeline dada para el problema de clasificación como exactitud, average_precision, roc_auc, recall, etc. El valor por defecto es accuracy.

- cv: Estrategia de validación cruzada utilizada al evaluar las tuberías. El valor predeterminado es 5.

- random_state: La semilla del generador de números pseudoaleatorios utilizado en TPOT. 

Como nuestro objetivo es sólo ilustrar el uso de TPOT, hemos fijado el tiempo máximo de optimización en 2 minutos (max_time_mins=2). En un portátil estándar con 4GB de RAM, se tarda aproximadamente 5 minutos en ejecutarse por generación. Por cada generación añadida, debería tardar 5 minutos más. Por lo tanto, para el valor por defecto de 100, el tiempo total de ejecución podría ser aproximadamente de unas 8 horas.



In [12]:
from tpot import TPOTClassifier
from tpot import TPOTRegressor

tpot = TPOTClassifier(verbosity=2, max_time_mins=2, max_eval_time_mins=0.04, population_size=40)
tpot.fit(df_new[training_indices], titanic_class[training_indices])

/opt/conda/lib/python3.8/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")


Optimization Progress:   0%|          | 0/40 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.8053192683200538

Generation 2 - Current best internal CV score: 0.8069128043990574

2.00 minutes have elapsed. TPOT will close down.
TPOT closed during evaluation in one generation.


TPOT closed prematurely. Will use the current best pipeline.

Best pipeline: RandomForestClassifier(GaussianNB(input_matrix), bootstrap=True, criterion=gini, max_features=1.0, min_samples_leaf=6, min_samples_split=4, n_estimators=100)


TPOTClassifier(max_eval_time_mins=0.04, max_time_mins=2, population_size=40,
               verbosity=2)

In [13]:
tpot.score(df_new[validation_indices], df.loc[validation_indices, 'class'].values)


0.820627802690583

TPOT, por lo general, lleva una gran cantidad de tiempo en ejecutarse. Con la configuración predeterminada de TPOT (100 generaciones con 100 tamaños de población), TPOT evaluará 10.000 configuraciones diferentes antes de terminar. Se trata de 10.000 configuraciones de modelos con una validación cruzada de 10, lo que significa que aproximadamente 100.000 modelos se ajustan y evalúan para los datos suministrados. Es un procedimiento que lleva mucho tiempo, incluso para modelos más sencillos como los árboles de decisión.

En el siguiente ejemplo, se ejecutan hasta 5 generaciones.

In [14]:
tpot = TPOTClassifier(generations=5, verbosity=2)
tpot.fit(df_new[training_indices], titanic_class[training_indices])

Optimization Progress:   0%|          | 0/600 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.817383009763214

Generation 2 - Current best internal CV score: 0.817383009763214

Generation 3 - Current best internal CV score: 0.817383009763214

Generation 4 - Current best internal CV score: 0.817383009763214

Generation 5 - Current best internal CV score: 0.817383009763214

Best pipeline: GradientBoostingClassifier(input_matrix, learning_rate=0.1, max_depth=4, max_features=0.5, min_samples_leaf=12, min_samples_split=19, n_estimators=100, subsample=1.0)


TPOTClassifier(generations=5, verbosity=2)

In [15]:
tpot.score(df_new[validation_indices], df.loc[validation_indices, 'class'].values)


0.820627802690583

Una de las funcionales que distinguen a TPOT, es la capacidad de exportar el código Python correspondiente a la pipeline óptima

In [10]:
tpot.export('pipeline.py')

## Ventajas:

Fácil de usar y de instalar
Entranamiento en paralelo con Dask
Soporta redes neuronales como una característica experimental 


## Contras:

TPOT puede tardar mucho tiempo en terminar su búsqueda. Ejecutar el TPOT no solo ajusta un modelo a un conjunto de datos. Utiliza múltiples algoritmos de aprendizaje automático (random forests, geométricos, SVMs, etc.) en una cadena con numerosos pasos de preprocesamiento (imputación de valores perdidos, escalado, PCA, selección de características, etc.), los hiperparámetros para todos los modelos y pasos de preprocesamiento, así como múltiples formas de ensamblar o apilar los algoritmos dentro de la cadena. Por ello, suele tardar mucho tiempo en ejecutarse y no es factible para grandes conjuntos de datos.

TPOT puede recomendar diferentes modelos para el mismo conjunto de datos. Si se utiliza un conjunto de datos razonablemente complejo o se ejecuta el TPOT durante un corto período de tiempo, realizar diferentes ejecuciones de TPOT pueden dar lugar a distintas recomendaciones. Cuando dos ejecuciones de TPOT recomiendan diferentes pipelines, esto significa que no convergieron debido a la falta de tiempo o que múltiples pipelines funcionan más o menos igual en el conjunto de datos.